In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import akshare as ak
import datetime
from datetime import date, timedelta
import seaborn as sns
import time


from fund_tools import *

In [2]:
# Set date variable

today = date.today()
first_day_of_the_week = today - timedelta(days=today.weekday())
# end = start + timedelta(days=6)
print("Today: " + str(today))
print("Start: " + str(first_day_of_the_week))
# print("End: " + str(end))

Today: 2021-09-30
Start: 2021-09-27


In [3]:
start_date = '2015-01-01'
end_date = today

fund_net_value = get_fund_net_worth('001980',start_date=start_date, end_date=end_date, fund_category='open')

In [150]:
def AIP_weekly(code, start_date, end_date, fund_category, fixed_investment, freq='Monday'):
    
    
    fund_net_value = get_fund_net_worth(code, start_date=start_date, end_date=end_date, fund_category=fund_category)
    
    fund_net_value['WeekDay'] = pd.to_datetime(fund_net_value['净值日期']).dt.day_name()
    
    fund_net_value['定投金额(本金)'] = 0
        
    for i in range(len(fund_net_value['WeekDay'])):
         if fund_net_value['WeekDay'].values[i] == freq:
                fund_net_value['定投金额(本金)'][i] = fixed_investment
            
    fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()
    fund_net_value['购买份额'] = fund_net_value['定投金额(本金)']/fund_net_value['单位净值']
    fund_net_value['累计份额'] = fund_net_value['购买份额'].cumsum()
    fund_net_value['平均成本'] = fund_net_value['累计定投金额(本金)']/fund_net_value['累计份额']



    fund_net_value['累计收益'] = (fund_net_value['单位净值'] - fund_net_value['平均成本']) * fund_net_value['累计份额']
    fund_net_value['持有天数'] = (fund_net_value['净值日期'] - fund_net_value['净值日期'][0]).dt.days+1
    fund_net_value['年化收益率'] = ((fund_net_value['累计收益'] + fund_net_value['累计定投金额(本金)'])/fund_net_value['累计定投金额(本金)'])**(365/fund_net_value['持有天数'])-1

    fund_net_value['累计收益率'] = fund_net_value['累计收益']/fund_net_value['累计定投金额(本金)']
    
    Stat_df = pd.DataFrame({
        '基金代码': code,
        '持有天数': fund_net_value['持有天数'].values[-1],
        '定投时间': freq,
        '定投金额': fixed_investment,
        '分投期数': fund_net_value['累计定投金额(本金)'].values[-1]/fixed_investment,
        '总购买份额' : '%.3f' % fund_net_value['累计份额'].values[-1],
        '平均成本' : '%.3f' % fund_net_value['平均成本'].values[-1],
        '累计收益' : '%.3f' % fund_net_value['累计收益'].values[-1],
        '累计收益率' : '%.3f' % fund_net_value['累计收益率'].values[-1],
        '年化收益率' : '%.3f' % fund_net_value['年化收益率'].values[-1]
    }, index=['Statistics'])
    
    return Stat_df

In [151]:
AIP_weekly('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, freq='Monday')

,基金代码,持有天数,定投时间,定投金额,分投期数,总购买份额,平均成本,累计收益,累计收益率,年化收益率
Statistics,001980,1234,Monday,1000,158.0,138466.734,1.141,106596.083,0.675,0.165


In [138]:
Freq = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

for freq in Freq:
    print(AIP_weekly('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, freq=freq))

                    总购买份额      平均成本           累计收益     累计收益率     年化收益率
Statistics  138466.734499  1.141068  106596.082954  0.674659  0.164754
                    总购买份额      平均成本           累计收益     累计收益率     年化收益率
Statistics  141338.482931  1.146185  108083.707034  0.667183  0.163214
                    总购买份额      平均成本           累计收益     累计收益率     年化收益率
Statistics  143250.605325  1.144847  109737.581715  0.669132  0.163616
                    总购买份额      平均成本           累计收益     累计收益率     年化收益率
Statistics  143136.130293  1.145762  109518.831378  0.667798  0.163341
                    总购买份额      平均成本           累计收益     累计收益率     年化收益率
Statistics  141864.342764  1.141936  109088.572588  0.673386  0.164492


In [67]:
fund_net_value['定投金额(本金)'] = 100
fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()
fund_net_value['购买份额'] = fund_net_value['定投金额(本金)']/fund_net_value['单位净值']
fund_net_value['累计份额'] = fund_net_value['购买份额'].cumsum()
fund_net_value['平均成本'] = fund_net_value['累计定投金额(本金)']/fund_net_value['累计份额']

fund_net_value['累计收益'] = (fund_net_value['单位净值'] - fund_net_value['平均成本']) * fund_net_value['累计份额']
fund_net_value['持有天数'] = (fund_net_value['净值日期'] - fund_net_value['净值日期'][0]).dt.days
fund_net_value['年化收益率'] = ((fund_net_value['累计收益'] + fund_net_value['累计定投金额(本金)'])/fund_net_value['累计定投金额(本金)'])**(365/fund_net_value['持有天数'])-1
fund_net_value['累计收益率'] = fund_net_value['累计收益']/fund_net_value['累计金额']


# average_cost = fund_net_value['累计金额'][-1]/fund_net_value['累计份额'][-1]


fund_net_value

,净值日期,单位净值,日增长率,定投金额,累计金额,购买份额,累计份额,平均成本,累计收益,累计收益率,定投金额(本金),累计定投金额(本金),持有天数,年化收益率
0,2018-05-16,1.0000,0.00,1000,1000,100.000000,100.000000,1.000000,0.000000,0.000000,100,100,0,0.000000
1,2018-05-18,1.0002,0.00,1000,2000,99.980004,199.980004,1.000100,0.020000,0.000010,100,200,2,0.018417
2,2018-05-25,1.0003,0.00,1000,3000,99.970009,299.950013,1.000167,0.039998,0.000013,100,300,9,0.005421
3,2018-06-01,0.9999,0.00,1000,4000,100.010001,399.960014,1.000100,-0.079982,-0.000020,100,400,16,-0.004552
4,2018-06-08,1.0008,0.00,1000,5000,99.920064,499.880078,1.000240,0.279982,0.000056,100,500,23,0.008923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,2021-09-23,1.9879,-0.13,1000,807000,50.304341,70742.311431,1.140760,59928.640894,0.074261,100,80700,1226,0.179802
807,2021-09-24,1.9603,-1.39,1000,808000,51.012600,70793.324031,1.141351,57976.153099,0.071753,100,80800,1227,0.174566
808,2021-09-27,1.9263,-1.73,1000,809000,51.912994,70845.237025,1.141926,55569.180082,0.068689,100,80900,1230,0.167851
809,2021-09-28,1.9249,-0.07,1000,810000,51.950751,70897.187776,1.142499,55469.996750,0.068481,100,81000,1231,0.167278
